In [1]:
# This script is for loading outputs from LIOSAM and ORBSLAM, and convert them to the file as the input of read_pose

In [2]:
import os

working_directory = "/home/charles/Documents/zhongnan/fastlio-color/test-offline-color/test-new-extrinsic"
# working_directory = "/data/BSD_data/Daiwu_sample"

config_file = os.path.join(working_directory, "index.json")
path_to_raw_image_folder = os.path.join(working_directory, "raw_images")
path_to_selected_image_folder = os.path.join(working_directory, "selected_images")

path_to_vo_file =  os.path.join(working_directory, "vo_interpolated_odom.txt")
path_to_keyframe_vo_file =  os.path.join(working_directory, "vo_odom_keyframes.txt")
path_to_keyframe_vo_trans_file =  os.path.join(working_directory, "vo_odom_keyframes_transformed.txt")

# path_to_undistort_img = os.path.join(working_directory, "/undistort_images")# "../data/Daiwu_sample/undistort_images/"

pcd_file = os.path.join(working_directory, "scans-clean-mls-clean.pcd")# "../data/Daiwu_sample/mesh.ply"
mesh_file = os.path.join(working_directory, "scans_mesh.ply")# "../data/Daiwu_sample/mesh.ply"

# trajectory_file =  os.path.join(working_directory, "vo_pano.txt")#"../data/Daiwu_sample/vo_pano.txt"

# mvs_pose_result =  os.path.join(working_directory, "mvs_pose_result.txt")#"../data/Daiwu_sample/mvs_pose_result.txt"

mvs_frame_result = os.path.join(working_directory, "mvs_frame_result.txt")#"../data/Daiwu_sample/mvs_frame_result.txt"


Load image and calibration file, then undistort fisheye images and save into "path_to_undistort_img"

In [3]:
import json

def load_calib(filename):
    # Load the json file
    with open(filename, 'r') as f:
        data = json.load(f)

    # Get the image resolution width and height
    width = data['camera_para']['w']
    height = data['camera_para']['h']
    # Get the camera intrinsic data
    fx = data['camera_para']['fx']
    fy = data['camera_para']['fy']
    cx = data['camera_para']['cx']
    cy = data['camera_para']['cy']
    k1 = data['camera_para']['k1']
    k2 = data['camera_para']['k2']
    k3 = data['camera_para']['k3']
    k4 = data['camera_para']['k4']

    t_LC = data["Tlc"]
    t_ci = data["Tci"]

    return width, height, fx, fy, cx, cy, k1, k2, k3, k4, t_LC, t_ci

In [4]:
import cv2
import numpy as np
import os
import shutil
from tqdm import tqdm

# Function to correct distortion for pinhole camera images
def undistort_image(img, k_matrix, distortion_coeffs):
    h, w = img.shape[:2]
    # Obtain the optimal new camera matrix based on the free scaling parameter (alpha)
    # Alpha=0 means all distorted points will be corrected but with some black pixels
    # Alpha=1 retains all original pixels but will also keep some distortion
    distortion_coeffs = np.array(distortion_coeffs)  # distortion coefficients
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(k_matrix, distortion_coeffs, (w, h), 1, (w, h))

    # Undistort the image
    undistorted_img = cv2.undistort(img, k_matrix, distortion_coeffs, None, new_camera_matrix)

    return undistorted_img, new_camera_matrix



In [5]:
# read vo from vo_file, then select keyframes from vo_file and save them into vo_keyframe_file
import math

def select_keyframe_from_vo(vo_file, vo_keyframe_file, threshold=0.5):
    # Read VO pano
    with open(vo_file, 'r') as f:
        vo_pano = f.readlines()

    keyframes = []
    last_position = None

    for line in vo_pano:
        data = line.strip().split()
        time = data[0]
        x, y, z = float(data[1]), float(data[2]), float(data[3])

        if last_position is None:
            keyframes.append(line)
            last_position = (x, y, z)
        else:
            distance = math.sqrt((x - last_position[0])**2 + (y - last_position[1])**2 + (z - last_position[2])**2)
            if distance >= threshold:
                keyframes.append(line)
                last_position = (x, y, z)

    # Save keyframes into vo_keyframe_file
    with open(vo_keyframe_file, 'w') as f:
        f.writelines(keyframes)

select_keyframe_from_vo(path_to_vo_file, path_to_keyframe_vo_file, threshold=0.5)



In [6]:
# Read images from image folder, then read keyframe info from vo_pano_file, then select keyframe images from image_folder,and store into output_folder
import cv2
import numpy as np
import os
import shutil
from tqdm import tqdm

def process_images(raw_image_folder, vo_keyframe_file, camera_intrinsics, selected_image_folder):
    # Camera intrinsics
    fx, fy, cx, cy, k1, k2, p3, p4 = camera_intrinsics

    # Create output folder if not exists
    if os.path.exists(selected_image_folder):
        shutil.rmtree(selected_image_folder)
    os.mkdir(selected_image_folder)


    # Read VO pano
    with open(vo_keyframe_file, 'r') as f:
        vo_pano = f.readlines()

    # Extract timestamps
    timestamps = [float(line.split()[0]) for line in vo_pano]

    image_files = sorted(os.listdir(raw_image_folder))

    # For each image in folder
    cnt = 0
    for idx, image_name in tqdm(enumerate(image_files), total=len(image_files), desc='Processing keyframes'):
        # Extract timestamp from image name
        timestamp = float(image_name.rsplit('.', 1)[0])
        
        # If timestamp is in vo_pano
        if timestamp in timestamps:
            # Load image
            img = cv2.imread(os.path.join(raw_image_folder, image_name))
            selected_image = os.path.join(selected_image_folder, str(f"{timestamp:.6f}") + ".jpg")
            
            # undistorted_img = undistort_image(img, fx, fy, cx, cy, k1, k2, k3, k4)
            k_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])  # Replace with your camera matrix values
            distortion_coeffs = [k1, k2, p3, p4]  # Replace with your distortion coefficients
            undistorted_img, new_intrinsic = undistort_image(img, k_matrix, distortion_coeffs)

            # Save image
            cv2.imwrite(selected_image, undistorted_img)   
            cnt += 1

    return new_intrinsic

In [7]:
width, height, fx, fy, cx, cy, k1, k2, k3, k4, t_LC, tci = load_calib(config_file)
cam_intrinsic = [ fx, fy, cx, cy, k1, k2, k3, k4]
print('Image Resolution - Width: {}, Height: {}'.format(width, height))
print('Camera Intrinsic Data - fx: {}, fy: {}, cx: {}, cy: {}'.format(fx, fy, cx, cy))
print(path_to_selected_image_folder)
new_intrtinsic = process_images(path_to_raw_image_folder, 
                                path_to_keyframe_vo_file,
                                cam_intrinsic,
                                path_to_selected_image_folder)
print("New camera intrinsic matrix is: {}".format(new_intrtinsic))
new_fx, new_fy, new_cx, new_cy = [new_intrtinsic[0,0],new_intrtinsic[1,1],new_intrtinsic[0,2],new_intrtinsic[1,2] ]


Image Resolution - Width: 4096, Height: 3000
Camera Intrinsic Data - fx: 4818.200388954926, fy: 4819.10345841615, cx: 2032.4178620390019, cy: 1535.1895959282901
/home/charles/Documents/zhongnan/fastlio-color/test-offline-color/test-new-extrinsic/selected_images


Processing keyframes: 100%|██████████| 588/588 [00:05<00:00, 117.59it/s]

New camera intrinsic matrix is: [[4.83111156e+03 0.00000000e+00 2.03085034e+03]
 [0.00000000e+00 4.82355224e+03 1.53477753e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [8]:
# transform lidar pose to camera pose 
import open3d as o3d
from scipy.spatial.transform import Rotation as R
import numpy as np

def transform_lo_to_vo(path_to_keyframe_vo_file, path_to_keyframe_vo_trans_file, t_LC, t_ci):
    t_LC = np.array(t_LC).reshape(4, 4)
    t_CL = np.linalg.inv(t_LC)
    t_ci = np.array(t_ci).reshape(4,4)
    t_ic = np.linalg.inv(t_ci)

    # print(t_LC)
    # print(t_CL)
    transformed_keyframes = []

    # Read VO pano
    with open(path_to_keyframe_vo_file, 'r') as f:
        vo_pano = f.readlines()

    for line in vo_pano:
        data = line.strip().split()
        time = data[0]
        x, y, z = float(data[1]), float(data[2]), float(data[3])
        qx, qy, qz, qw = [float(d) for d in data[4:]]
        
        # Convert LiDAR position and quaternion to pose matrix
        # lid_quaternion = [qw, qx, qy, qz]
        lid_quaternion = [ qx, qy, qz, qw]
        lid_rotation = R.from_quat(lid_quaternion).as_matrix()
        lid_pose = np.eye(4)
        lid_pose[:3, :3] = lid_rotation
        lid_pose[:3, 3] = [x, y, z]

        # 构造绕Z轴旋转180度的静态变换矩阵
        theta = np.pi  # 180度转换为弧度
        c, s = np.cos(theta), np.sin(theta)
        static_trans = np.array([[c, -s, 0, 0],
                                [s, c, 0, 0],
                                [0, 0, 1, 0],
                                [0, 0, 0, 1]])
                                
        
        # Transform LiDAR pose to Camera pose
        # cam_pose = np.dot(t_LC, lid_pose)
        # cam_pose = np.dot(lid_pose, t_LC)
        # cam_pose = np.dot(lid_pose, t_LC) 
        
        cam_pose = lid_pose


        # cam_pose = np.dot(np.dot(t_CL, lid_pose), static_trans)

        # Extract camera position and rotation matrix
        cam_position = cam_pose[:3, 3]
        cam_rotation_matrix = cam_pose[:3, :3]

        # Convert camera rotation matrix to quaternion
        cam_quaternion = R.from_matrix(cam_rotation_matrix).as_quat() # (x,y,z,w)
        # Format the transformed pose for saving
        transformed_line = f"{time} {cam_position[0]} {cam_position[1]} {cam_position[2]} {cam_quaternion[0]} {cam_quaternion[1]} {cam_quaternion[2]} {cam_quaternion[3]}\n"
        transformed_keyframes.append(transformed_line)

    # Save transformed keyframes into the specified file
    with open(path_to_keyframe_vo_trans_file, 'w') as f:
        f.writelines(transformed_keyframes)

# transform_lo_to_vo(path_to_keyframe_vo_file, path_to_keyframe_vo_trans_file, t_LC, tci)

Extract point info from ply file
For each point, calculate how many keyframe images observed this point, and record the id of these keyframe images

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from loguru import logger as logger
import open3d as o3d
from tqdm import tqdm


#  Visualization using Open3D
def visualize_point_cloud_and_cameras(vertices, rotation_matrix, camera_pose ):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(vertices)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    camera = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)
    world_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.8)

    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = np.array([camera_pose[0], camera_pose[1],camera_pose[2]])

    camera.transform(transformation_matrix)
    vis.add_geometry(camera)
    vis.add_geometry(world_origin)

    vis.run()
    vis.destroy_window()



In [10]:
import numpy as np
import open3d as o3d
from loguru import logger as logger
from tqdm import tqdm

# default FOV: vertical_fov=84.8, horizontal_fov=98.4
# vertical_fov=54.8, horizontal_fov=58.4
def calculate_keyframes(mesh_file, trajectory_file, t_LC, mvs_pose_result_file, vertical_fov=54.8, horizontal_fov=58.4):
    # 读取mesh.ply文件
    # mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh = o3d.io.read_point_cloud(mesh_file)

    vertices = np.asarray(mesh.points)
    logger.info("Vertices number in mesh.ply is {}".format(len(vertices)))

    # 读取视觉里程计轨迹
    trajectories = np.loadtxt(trajectory_file)

    # Read transformation matric from camera to lidar
    # t_LC = np.array(t_LC).reshape(4, 4)

    # # Extract rotation matrix and translation vector
    # R = t_LC[:3, :3]
    # t = t_LC[:3, 3]

    # # Calculate inverse transformation matrix from camera to LiDAR coordinates
    # Rt = np.transpose(R)
    # t_inv = -np.dot(Rt, t)
    # t_CL = np.eye(4)  # Initialize a 4x4 identity matrix
    # t_CL[:3, :3] = Rt
    # t_CL[:3, 3] = t_inv
    
    # 初始化结果
    result = []
    num_of_visible_point = 0

    # 初始化一个字典来存储每个点的关键帧ID
    point_to_keyframes = {i: [] for i in range(len(vertices))}

    # 使用hidden point removal计算在该viewpoint位置处，可见的点云
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(vertices)
    # transform the pcd object from lidar coord to camera coord by dotting the TF
    # pcd.transform(t_LC)
    # o3d.visualization.draw([pcd])

    diameter = np.linalg.norm(np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
    radius = diameter * 100
    logger.info(" hidden point removal radius is {}".format(radius))

    # 在转换点之前，创建一个索引映射
    index_map = np.arange(len(vertices))


    # 为单行情况创建一个虚拟的外层列表，使其与多行情况的处理逻辑保持一致
    if len(trajectories.shape) == 1:
        trajectories = [trajectories]
        
    # camera_poses = []
    # 遍历每个关键帧
    for idx, trajectory in tqdm(enumerate(trajectories), total=len(trajectories), desc='Processing keyframes'): # 获取相机姿态
        # timestamp = trajectory[0]
        camera_position = trajectory[1:4]
        # qx, qy, qz, qw = trajectory[4:]
        qw ,qx, qy, qz= trajectory[4:]
        # quaternion = [qw, qx, qy, qz]
        quaternion = [ qx, qy, qz, qw]
        # 将四元数转换为旋转矩阵
        camera_rotation = R.from_quat(quaternion).as_matrix() # rotation matrix from frame 0 to
        camera_pose = np.eye(4)
        camera_pose[:3, :3] = camera_rotation
        camera_pose[:3, 3] = camera_position
        camera_pose_inv = np.linalg.inv(camera_pose)
 
        # 使用hidden point removal计算在该viewpoint位置处，可见的点云
        _, pt_map = pcd.hidden_point_removal(camera_position, radius)

        # visible_points_pcd = pcd.select_by_index(pt_map)
        # o3d.io.write_point_cloud("visible_points.ply", visible_points_pcd)
        # o3d.visualization.draw([visible_points_pcd])

        visible_points = np.asarray(pcd.points)[pt_map]

        # 更新索引映射
        visible_indices = index_map[pt_map]

        # 使用相机pose以及相机的视场角，计算在该相机的pose处，可见的点云
        camera_z_direction = np.array([0,0,1])
        # camera_y_direction = np.array([0,1,0]) 

        visible_points_in_cam = camera_pose_inv[:3, :3] @  visible_points.transpose() + camera_pose_inv[:3, 3].reshape((3,1))
        visible_points_in_cam = visible_points_in_cam.transpose()

        visible_point_ranges = np.linalg.norm(visible_points_in_cam, axis=1)
        theta_z = np.arccos(np.dot(visible_points_in_cam, camera_z_direction) / visible_point_ranges)  # incident angle to camera frame
        zenith_angle = np.arccos(visible_points_in_cam[:,1] / visible_point_ranges)
        theta_z_in_deg = np.rad2deg(theta_z)
        zenith_angle_in_deg = np.rad2deg(zenith_angle)
        # logger.info("For idx {}, theta_z_in_deg is {}, zenith_angle_in_deg is {}".format(idx, theta_z_in_deg, zenith_angle_in_deg))
        # mask_fov = np.logical_and(zenith_angle < np.deg2rad(90 + vertical_fov/2), np.deg2rad(90 - vertical_fov/2) < zenith_angle, \
        #     theta_z < np.deg2rad(horizontal_fov / 2),  np.deg2rad(- horizontal_fov / 2) < theta_z )
        mask_zenith = np.logical_and(np.deg2rad(90 - vertical_fov/2) < zenith_angle, zenith_angle < np.deg2rad(90 + vertical_fov/2))
        mask_theta = np.logical_and(np.deg2rad(- horizontal_fov / 2) < theta_z, theta_z < np.deg2rad(horizontal_fov / 2))
        mask_fov = np.logical_and(mask_zenith, mask_theta)

        # 对该view里面可见的点，添加该keyframe为能看到此点的关键帧
        visible_points_in_fov = visible_points[mask_fov]
       
        # 对该view里面可见的点，添加该keyframe为能看到此点的关键帧
        for i, point in enumerate(visible_points_in_fov):
            original_index = visible_indices[i]
            point_to_keyframes[original_index].append(idx)

        # if visualize
        # visualize_point_cloud_and_cameras(visible_points, camera_rotation, camera_position)
        # visualize_point_cloud_and_cameras(visible_points_in_fov, camera_rotation, camera_position)


        # for point in visible_points:
        #     point_to_keyframes[np.where((vertices == point).all(axis=1))[0][0]].append(idx)

    # 计算结果
    logger.info("point_to_keyframes.items lens is {}".format(len(point_to_keyframes.items())))
    for point, keyframes in point_to_keyframes.items():
        num_of_keyframes = len(keyframes)
        if num_of_keyframes > 0:
            num_of_visible_point += 1
            result.append(f"{vertices[point][0]} {vertices[point][1]} {vertices[point][2]} {num_of_keyframes} {' '.join(map(str, keyframes))}")

    return vertices, result, num_of_visible_point

In [11]:
from tqdm import tqdm

trajectories = np.loadtxt(path_to_keyframe_vo_file)
# write MVS keyframe info
MVS_result = []
MVS_result.append(f"MVS {width} {height}")
MVS_result.append(f"{len(trajectories)}")

# 为单行情况创建一个虚拟的外层列表，使其与多行情况的处理逻辑保持一致
if len(trajectories.shape) == 1:
    trajectories = [trajectories]

for idx, trajectory in tqdm(enumerate(trajectories), total=len(trajectories), desc='Processing keyframes'): # 获取相机姿态
    print(trajectory)
    timestamp = trajectory[0]
    camera_position = trajectory[1:4]
    quaternion= trajectory[4:]
    # quaternion = [ qx, qy, qz,qw]

    # 将四元数转换为旋转矩阵
    camera_rotation = R.from_quat(quaternion).as_matrix() # rotation matrix from frame 0 to
    camera_pose = np.eye(4)
    camera_pose[:3, :3] = camera_rotation
    camera_pose[:3, 3] = camera_position
    camera_pose_inv = np.linalg.inv(camera_pose)

    camera_odom_cw_quat = R.from_matrix(camera_pose_inv[:3, :3]).as_quat() # x,y,z,w
    camera_odom_cw_pos = camera_pose_inv[:3, 3]
    [qx, qy, qz, qw] = camera_odom_cw_quat


    MVS_result.append(f"{idx} {new_fx} {new_fy} {new_cx} {new_cy} {qw} {qx} {qy} {qz} {camera_position[0]} {camera_position[1]} {camera_position[2]} {timestamp:6f}")

    # MVS_result.append(f"{idx} {new_fx} {new_fy} {new_cx} {new_cy} {qw} {qx} {qy} {qz} {camera_odom_cw_pos[0]} {camera_odom_cw_pos[1]} {camera_odom_cw_pos[2]} {timestamp:6f}")

with open(mvs_frame_result, 'w') as file:
    # file.write(f"{len(vertices)}\n")
    file.write("\n".join(MVS_result))


Processing keyframes: 100%|██████████| 27/27 [00:00<00:00, 10628.46it/s]

[ 1.71083328e+09 -1.88346820e-01  8.62947200e-02  9.96650000e-04
  6.99219150e-01 -7.14673180e-01  1.79808000e-03 -1.82100000e-02]
[ 1.71083328e+09 -1.97910160e-01 -1.27014560e-01  4.73792230e-01
  5.98587410e-01 -7.66460880e-01  1.46331000e-01 -1.81157580e-01]
[ 1.71083329e+09 -3.95816590e-01 -5.88155010e-01  4.24732900e-01
  6.03932710e-01 -7.66056570e-01  1.45930110e-01 -1.64702850e-01]
[ 1.7108333e+09  5.1838530e-02 -8.0528458e-01  6.2191405e-01
  5.2992508e-01 -8.4769562e-01 -1.6161050e-02 -1.8175880e-02]
[ 1.7108333e+09  5.5620274e-01 -7.2938203e-01  6.4741174e-01
  5.2538298e-01 -8.1699381e-01 -2.1056237e-01  1.1026026e-01]
[ 1.71083330e+09  1.04930114e+00 -5.37403170e-01  6.70148700e-01
  4.90965080e-01 -7.92224780e-01 -3.03791220e-01  1.97595770e-01]
[ 1.71083331e+09  1.39951189e+00 -1.51828650e-01  6.33210510e-01
  4.55545060e-01 -7.16607410e-01 -4.42014480e-01  2.89094640e-01]
[ 1.71083331e+09  1.50027147e+00  3.44898030e-01  5.83153860e-01
  4.27129360e-01 -5.85226010e-01 -

In [12]:
transform_matrix_LC = t_LC
vertices, result, num_of_visible_point = calculate_keyframes(pcd_file, path_to_keyframe_vo_file, transform_matrix_LC, mvs_frame_result)

# 写入结果到txt文件
with open(mvs_frame_result, 'a') as file:
    file.write("\n")
    file.write(f"{num_of_visible_point}\n")
    file.write("\n".join(result))

print(f"Results appended to {mvs_frame_result}")
print(f"num_of_visible_point {num_of_visible_point}")

AttributeError: 'open3d.cpu.pybind.geometry.PointCloud' object has no attribute 'vertices'